## sklearn and TextAttack

This following code trains two different text classification models using sklearn. Both use logistic regression models: the difference is in the features. 

We will load data using `datasets`, train the models, and attack them using TextAttack.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_1_sklearn.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_1_sklearn.ipynb)

In [1]:
!pip install datasets nltk sklearn

Please remember to run  **pip3 install textattack[tensorflow]**  in your notebook enviroment before the following codes:

### Training

This code trains two models: one on bag-of-words statistics (`bow_unstemmed`) and one on tf–idf statistics (`tfidf_unstemmed`). The dataset is the IMDB movie review dataset.

In [2]:
import nltk  # the Natural Language Toolkit

nltk.download("punkt")  # The NLTK tokenizer

In [3]:
import datasets
import os
import pandas as pd
import re
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Nice to see additional metrics
from sklearn.metrics import classification_report


def load_data(dataset_split="train"):
    dataset = datasets.load_dataset("rotten_tomatoes")[dataset_split]
    # Open and import positve data
    df = pd.DataFrame()
    df["Review"] = [review["text"] for review in dataset]
    df["Sentiment"] = [review["label"] for review in dataset]
    # Remove non-alphanumeric characters
    df["Review"] = df["Review"].apply(lambda x: re.sub("[^a-zA-Z]", " ", str(x)))
    # Tokenize the training and testing data
    df_tokenized = tokenize_review(df)
    return df_tokenized


def tokenize_review(df):
    # Tokenize Reviews in training
    tokened_reviews = [word_tokenize(rev) for rev in df["Review"]]
    # Create word stems
    stemmed_tokens = []
    porter = PorterStemmer()
    for i in range(len(tokened_reviews)):
        stems = [porter.stem(token) for token in tokened_reviews[i]]
        stems = " ".join(stems)
        stemmed_tokens.append(stems)
    df.insert(1, column="Stemmed", value=stemmed_tokens)
    return df


def transform_BOW(training, testing, column_name):
    vect = CountVectorizer(
        max_features=100, ngram_range=(1, 3), stop_words=ENGLISH_STOP_WORDS
    )
    vectFit = vect.fit(training[column_name])
    BOW_training = vectFit.transform(training[column_name])
    BOW_training_df = pd.DataFrame(
        BOW_training.toarray(), columns=vect.get_feature_names()
    )
    BOW_testing = vectFit.transform(testing[column_name])
    BOW_testing_Df = pd.DataFrame(
        BOW_testing.toarray(), columns=vect.get_feature_names()
    )
    return vectFit, BOW_training_df, BOW_testing_Df


def transform_tfidf(training, testing, column_name):
    Tfidf = TfidfVectorizer(
        ngram_range=(1, 3), max_features=100, stop_words=ENGLISH_STOP_WORDS
    )
    Tfidf_fit = Tfidf.fit(training[column_name])
    Tfidf_training = Tfidf_fit.transform(training[column_name])
    Tfidf_training_df = pd.DataFrame(
        Tfidf_training.toarray(), columns=Tfidf.get_feature_names()
    )
    Tfidf_testing = Tfidf_fit.transform(testing[column_name])
    Tfidf_testing_df = pd.DataFrame(
        Tfidf_testing.toarray(), columns=Tfidf.get_feature_names()
    )
    return Tfidf_fit, Tfidf_training_df, Tfidf_testing_df


def add_augmenting_features(df):
    tokened_reviews = [word_tokenize(rev) for rev in df["Review"]]
    # Create feature that measures length of reviews
    len_tokens = []
    for i in range(len(tokened_reviews)):
        len_tokens.append(len(tokened_reviews[i]))
    len_tokens = preprocessing.scale(len_tokens)
    df.insert(0, column="Lengths", value=len_tokens)

    # Create average word length (training)
    Average_Words = [len(x) / (len(x.split())) for x in df["Review"].tolist()]
    Average_Words = preprocessing.scale(Average_Words)
    df["averageWords"] = Average_Words
    return df


def build_model(X_train, y_train, X_test, y_test, name_of_test):
    log_reg = LogisticRegression(C=30, max_iter=200).fit(X_train, y_train)
    y_pred = log_reg.predict(X_test)
    print(
        "Training accuracy of " + name_of_test + ": ", log_reg.score(X_train, y_train)
    )
    print("Testing accuracy of " + name_of_test + ": ", log_reg.score(X_test, y_test))
    print(classification_report(y_test, y_pred))  # Evaluating prediction ability
    return log_reg


# Load training and test sets
# Loading reviews into DF
df_train = load_data("train")

print("...successfully loaded training data")
print("Total length of training data: ", len(df_train))
# Add augmenting features
df_train = add_augmenting_features(df_train)
print("...augmented data with len_tokens and average_words")

# Load test DF
df_test = load_data("test")

print("...successfully loaded testing data")
print("Total length of testing data: ", len(df_test))
df_test = add_augmenting_features(df_test)
print("...augmented data with len_tokens and average_words")

# Create unstemmed BOW features for training set
unstemmed_BOW_vect_fit, df_train_bow_unstem, df_test_bow_unstem = transform_BOW(
    df_train, df_test, "Review"
)
print("...successfully created the unstemmed BOW data")

# Create TfIdf features for training set
unstemmed_tfidf_vect_fit, df_train_tfidf_unstem, df_test_tfidf_unstem = transform_tfidf(
    df_train, df_test, "Review"
)
print("...successfully created the unstemmed TFIDF data")

# Running logistic regression on dataframes
bow_unstemmed = build_model(
    df_train_bow_unstem,
    df_train["Sentiment"],
    df_test_bow_unstem,
    df_test["Sentiment"],
    "BOW Unstemmed",
)

tfidf_unstemmed = build_model(
    df_train_tfidf_unstem,
    df_train["Sentiment"],
    df_test_tfidf_unstem,
    df_test["Sentiment"],
    "TFIDF Unstemmed",
)

### Attacking

TextAttack includes a build-in `SklearnModelWrapper` that can run attacks on most sklearn models. (If your tokenization strategy is different than above, you may need to subclass `SklearnModelWrapper` to make sure the model inputs & outputs come in the correct format.)

Once we initializes the model wrapper, we load a few samples from the IMDB dataset and run the `TextFoolerJin2019` attack on our model.

In [4]:
from textattack.models.wrappers import SklearnModelWrapper

model_wrapper = SklearnModelWrapper(bow_unstemmed, unstemmed_BOW_vect_fit)

In [5]:
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker

dataset = HuggingFaceDataset("rotten_tomatoes", None, "train")
attack = TextFoolerJin2019.build(model_wrapper)

attacker = Attacker(attack, dataset)
attacker.attack_dataset()

### Conclusion
We were able to train a model on the IMDB dataset using `sklearn` and use it in TextAttack by initializing with the `SklearnModelWrapper`. It's that simple!